In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, f1_score
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset



# Encode labels
label_encoder = LabelEncoder()
df['emotion_encoded'] = label_encoder.fit_transform(df['emotion'])
val_df['emotion_encoded'] = label_encoder.transform(val_df['emotion'])

# Load Hinglish RoBERTa tokenizer and model for sequence classification
tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/hing-roberta")
model = AutoModelForSequenceClassification.from_pretrained("l3cube-pune/hing-roberta", num_labels=8)  # Set num_labels to the number of emotion classes

# Tokenize and prepare data
def tokenize_data(data, tokenizer, max_length=128):
    tokenized_data = tokenizer(data['utterance'].tolist(), truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')
    return tokenized_data

train_tokenized = tokenize_data(df, tokenizer)
val_tokenized = tokenize_data(val_df, tokenizer)

# Convert labels to PyTorch tensors
y_train = torch.tensor(df['emotion_encoded'].values)
y_val = torch.tensor(val_df['emotion_encoded'].values)

# Create DataLoader for training and validation
train_dataset = TensorDataset(train_tokenized['input_ids'], train_tokenized['attention_mask'], y_train)
val_dataset = TensorDataset(val_tokenized['input_ids'], val_tokenized['attention_mask'], y_val)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Define training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()

num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_predictions = []
    val_true_labels = []

    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=1).cpu().numpy()

            val_predictions.extend(predictions)
            val_true_labels.extend(labels.cpu().numpy())

    val_accuracy = accuracy_score(val_true_labels, val_predictions)
    val_report = classification_report(val_true_labels, val_predictions)
    val_weighted_f1 = f1_score(val_true_labels, val_predictions, average='weighted')

    print(f"Epoch {epoch + 1}/{num_epochs}")
    print("Validation Accuracy:", val_accuracy)
    print("Validation Classification Report:\n", val_report)
    print("Validation Weighted F1 Score:", val_weighted_f1)
    print("-" * 50)


In [ ]:
import numpy as np

# Move the model to CPU
model.to("cpu")

# Assuming you have a test_df DataFrame with 'utterance' column
test_tokenized = tokenize_data(test_df, tokenizer)

# Convert test data to PyTorch tensors
test_dataset = TensorDataset(test_tokenized['input_ids'], test_tokenized['attention_mask'])
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

model.eval()
test_predictions = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask = batch

        # Ensure data is on the same device as the model
        input_ids, attention_mask = input_ids.to("cpu"), attention_mask.to("cpu")

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1).cpu().numpy()

        # Append predictions to the test_predictions list
        test_predictions.extend(predictions)

# Print the numeric predictions before mapping them back to labels
print("Numeric Predictions:", test_predictions)

# Map numeric predictions back to original labels using label_encoder.classes_
test_emotion_labels = label_encoder.classes_[test_predictions]

# Save results to a text file
with open('test_predictions_roberta.txt', 'w') as file:
    for label in test_emotion_labels:
        file.write(label + '\n')

print("Predictions saved to test_predictions.txt")


Hing Roberta Mixed

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, f1_score
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
from torch.nn.utils import clip_grad_norm_
from transformers import get_linear_schedule_with_warmup

# Assuming you have train_df and val_df DataFrames with 'utterance' and 'emotion' columns

# Encode labels
label_encoder = LabelEncoder()
df['emotion_encoded'] = label_encoder.fit_transform(df['emotion'])
val_df['emotion_encoded'] = label_encoder.transform(val_df['emotion'])

# Load Hinglish BERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/hing-roberta-mixed")
model = AutoModelForSequenceClassification.from_pretrained("l3cube-pune/hing-roberta-mixed", num_labels=len(label_encoder.classes_))

# Tokenize and prepare data
def tokenize_data(data, tokenizer, max_length=128):
    tokenized_data = tokenizer(data['utterance'].tolist(), truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')
    return tokenized_data

train_tokenized = tokenize_data(df, tokenizer)
val_tokenized = tokenize_data(val_df, tokenizer)

# Convert labels to PyTorch tensors
y_train = torch.tensor(df['emotion_encoded'].values)
y_val = torch.tensor(val_df['emotion_encoded'].values)

# Create DataLoader for training and validation
train_dataset = TensorDataset(train_tokenized['input_ids'], train_tokenized['attention_mask'], y_train)
val_dataset = TensorDataset(val_tokenized['input_ids'], val_tokenized['attention_mask'], y_val)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Define training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)  # Adjusted learning rate and added weight decay
criterion = CrossEntropyLoss()

total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        loss = criterion(logits, labels)
        loss.backward()

        # Implement gradient clipping
        clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        scheduler.step()

    # Validation
    model.eval()
    val_predictions = []
    val_true_labels = []

    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=1).cpu().numpy()

            val_predictions.extend(predictions)
            val_true_labels.extend(labels.cpu().numpy())

    val_accuracy = accuracy_score(val_true_labels, val_predictions)
    val_report = classification_report(val_true_labels, val_predictions)
    val_weighted_f1 = f1_score(val_true_labels, val_predictions, average='weighted')

    print(f"Epoch {epoch + 1}/{num_epochs}")
    print("Validation Accuracy:", val_accuracy)
    print("Validation Classification Report:\n", val_report)
    print("Validation Weighted F1 Score:", val_weighted_f1)
    print("-" * 50)
